# Overview

This competition is a joint initiative by Planet and SCCON, showcasing how satellite imagery can be leveraged to monitor deforestation and climate change overtime. In this challenge, we need to design a multilabel classifier to help detect both small and large scale interventionism within the Amazon forest.

The stakes of this project are paramount to share knowledge on computer vision applied to forestry and generalize these techniques to other forest monitoring use-cases. Datasets and AI projects on forests are still scarced, so there is a growing need to create massive catalog and gather intelligence about biodiversity. Ultimately, it will benefits and attract more company like [Pachama](https://pachama.com) or [Avalanche](https://avalanche.green/) to protect and get a deeper understanding of our forests.

The quantity of labelized images in this challenge is limited (about 40,000), so we need to retrain existing models on this small Planet dataset. Resnet has shown to be a strong candidate on previous challenges among pretrained models like VGG, Inception and DenseNet (full list of pretrained torchvision models [here](https://pytorch.org/vision/stable/models.html)). New deep learning techniques are emerging and ideas worth exploring include the following:
- [Cossim CNN](https://www.rpisoni.dev/posts/cossim-convolution/)
- [Vision Transformers](https://github.com/lucidrains/vit-pytorch)
- [Self-supervised learning](https://docs.lightly.ai/tutorials/package/tutorial_simsiam_esa.html)

This dataset initially contained 4 bands satellite imagery —R, G, B and Near-Infrared (NIR)—, available through torrent download. The torrents seem to have been disabled but fortunately, a Kaggler have saved [the entire RGB .png dataset](https://www.kaggle.com/nikitarom/planets-dataset). The .tif files containing the NIR dataset however, aren't available anymore. NIR is especially useful to compute greenness indexes such as the NDVI. It allows a finer visualisation of the tree canopee and measurement of the carbon absorption, although we would need to recreate a model from scratch to use it, since all models have been pretrained on RGB only.

The dataset contains a wide class imbalance, with primary forest presence on more than 90% of the labels, whereas conventional mining only account for 0.25%. We need to monitor the scores for these rare classes, as they are the key to an overall good score and an efficient model to detect ponctual human activity.

## Packages and setup

In [4]:
import os
import random
from time import time
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pickle

from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.metrics import fbeta_score, confusion_matrix

from matplotlib import pyplot as plt
%matplotlib inline

import cv2

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms as T, models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
!pip install -q torchsummary --user
!pip install gdown
from torchsummary import summary


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We set random seeding for reproducibility purposes.

In [6]:
random.seed(101)
np.random.seed(101)
torch.manual_seed(101);

In [7]:
!gdown https://drive.google.com/uc?id=1Rm2F5bhPqbRcLBNqOEL6kr4FKdKYun2T -O amazon-dataset-split-1.tar.gz # new split 1/3
!gdown https://drive.google.com/uc?id=1Qpdhlky4uaMBj4kvsKnFWD1nAsEaI5UH -O train_classes.csv
!tar -xzf amazon-dataset-split-1.tar.gz && rm -f amazon-dataset-split-1.tar.gz

Downloading...
From: https://drive.google.com/uc?id=1Rm2F5bhPqbRcLBNqOEL6kr4FKdKYun2T
To: /content/amazon-dataset-split-1.tar.gz
100% 292M/292M [00:01<00:00, 218MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Qpdhlky4uaMBj4kvsKnFWD1nAsEaI5UH
To: /content/train_classes.csv
100% 1.43M/1.43M [00:00<00:00, 165MB/s]


In [8]:
train_paths = ["amazon-dataset-split-1/train", ]
val_paths = ["amazon-dataset-split-1/validation", ]
num_train = sum([len(os.listdir(path)) for path in train_paths])
num_val = sum([len(os.listdir(path)) for path in val_paths])
print(
    f"train files: {num_train}, "
    f"validation files: {num_val}"
)

train files: 10635, validation files: 8127


In [9]:
path_class = os.path.join("train_classes.csv")
df_class = pd.read_csv(path_class)
df_class = df_class
df_class.head()
df_class["list_tags"] = df_class.tags.str.split(" ")

In [10]:
df_class.head()  # all possible train files

,image_name,tags,list_tags
0,train_0,haze primary,"[haze, primary]"
1,train_1,agriculture clear primary water,"[agriculture, clear, primary, water]"
2,train_2,clear primary,"[clear, primary]"
3,train_3,clear primary,"[clear, primary]"
4,train_4,agriculture clear habitation primary road,"[agriculture, clear, habitation, primary, road]"


# Model training

### Transformations

For optimal performances, [resnet18 need input shape that are multiple of 32](https://discuss.pytorch.org/t/input-image-shape-for-resnet18-architecture/80693) and in our case we have input of size 256. From 256, the closest multiple of 32 is 224.

Therefore, we rescale our input data using this multiple, and we also normalize our dataset based on resnet pretrained mean and standard deviation intensity values. `ToTensor()` is useful to normalize our image values from 0-255 range to 0-1 range.

In [11]:
def get_transforms():
    transform_train = T.Compose([
      T.ToPILImage(),
      T.Resize(224),
      T.ToTensor(),
      T.Normalize(
          mean=[0.485, 0.456, 0.406],  # Set by previously known values
          std=[0.229, 0.224, 0.225],   # Set by previously known values
      )
    ])
    transform_val = T.Compose([
      T.ToPILImage(),
      T.Resize(224),
      T.ToTensor(),
      T.Normalize(
        mean=[0.485, 0.456, 0.406],  # Set by previously known values
        std=[0.229, 0.224, 0.225],   # Set by previously known values
      )
    ])
    return transform_train, transform_val

We define our custom Dataset class to manipulate batches of data between RAM and Disk more easily. Some point of attentions:
- `__init__`: we pass the dataframe (wich also contains the target), the transformation, the file path(s) and `is_train` flag. It is important to distinguish the training phase from the testing phase because we use test augmentation. <br> Test augmentation (TTA) is helpful to diversify our training dataset and build a more robust model. It is applied on each image for each batch, meaning that is doesn't increase the length of our training dataset per say, but it transforms each image randomly during execution time.
- `__getitem__`: we define what the dataset return upon iteration. It needs to load both image and target.
- `collate_fn`: we use this function within the following DataLoader instance. It corresponds to the batch manipulation. This is were `transform` is called. We also proceed to train and test augmentation there.

In [12]:
class AmazonDataset(Dataset):
    def __init__(
        self,
        df,
        transform,
        paths,
        is_train=True,
        idx_tta=None,
    ):
        super().__init__()
        self.df = df
        self.ohe_tags = list(map(transform_targets, self.df.list_tags.values))
        self.transform = transform
        if isinstance(paths, str):
            self.paths = [paths]
        elif isinstance(paths, (list, tuple)):
            self.paths = paths
        else:
            raise ValueError('expected path to be a string or list or tuple')
        self.is_train = is_train
        if not is_train:
            if not idx_tta in list(range(6)):
                raise ValueError(
                    f"In test mode, 'idx_tta' must be an int belonging to [0, 5], got: {repr(idx_tta)}"
                )
            self.idx_tta = idx_tta

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filename = self.df.iloc[idx].image_name + ".jpg"
        files = list()
        for path in self.paths:
            file_path = os.path.join(path, filename)
            if os.path.isfile(file_path):
                files.append(file_path)
        if len(files) == 0:
            raise FileNotFoundError(filename + ' not found in ' + ' '.join(self.paths))
        elif len(files) > 1:
            raise ValueError(f'Found {len(files)} values for {filename} as {files}')
        else:
            file_path = files[0]
        img = cv2.imread(file_path)
        if img is None:
            raise ValueError('Problem reading image from %s' % file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.ohe_tags[idx]
        return img, label

    def collate_fn(self, batch):
        imgs, labels = [], []
        for (img, label) in batch:
            img = self.custom_augment(img)
            img = torch.tensor(img)
            img = img.permute(2, 0, 1)
            img = self.transform(img)
            imgs.append(img[None])
            labels.append(label)
        imgs = torch.cat(imgs).float().to(device)
        labels = torch.tensor(labels).float().to(device)
        return imgs, labels

    def load_img(self, idx, ax=None):
        img, ohe_label = self[idx]
        label = self.df.iloc[idx].tags
        title = f"{label} - {ohe_label}"
        if ax is None:
            plt.imshow(img)
            plt.title(title)
        else:
            ax.imshow(img)
            ax.set_title(title)

    def custom_augment(self, img):
        """
        Not used used in this notebook.
        """
        return img

We are now ready for all full data pipeline! We define our target encoder first before, wrapping our custom train and validation dataset within `DataLoaders`, with a batch size of 64 —a good trade-off between RAM usage and speed in our case.

In [13]:
LABEL_IDXS = {
    'agriculture': 0,
    'artisinal_mine': 1,
    'bare_ground': 2,
    'blooming': 3,
    'blow_down': 4,
    'clear': 5,
    'cloudy': 6,
    'conventional_mine': 7,
    'cultivation': 8,
    'habitation': 9,
    'haze': 10,
    'partly_cloudy': 11,
    'primary': 12,
    'road': 13,
    'selective_logging': 14,
    'slash_burn': 15,
    'water': 16,
}
INVERSE_LABEL_IDXS = {v: k for k, v in LABEL_IDXS.items()}
def transform_targets(tag_list):
    # tag_list is a list of words (subset of LABEL_IDXS)
    targets = np.zeros(len(LABEL_IDXS), dtype=np.int8)
    mask = [LABEL_IDXS[tag] for tag in tag_list]
    targets[mask] = 1
    # returns list of ones and zeros
    # (e.g. tag_list = ['primary', 'haze'] -> [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]
    return targets.tolist()
def inverse_transform_targets(targets):
    idxs = [idx for idx, target in enumerate(targets) if target]
    tag_list = [INVERSE_LABEL_IDXS[ix] for ix in idxs]
    return list(sorted(tag_list))

Sanity check: we expect `imgs` to be a batch of our chosen batch size (64), with 3 channels and of chosen image dimensions (224x224). <br>
`labels` are also a batch of our chosen size (64) with 17 different classes.

In [14]:
def load_split(paths, df_class):
    if isinstance(paths, str):
        paths = [paths]
    elif not isinstance(paths, (list, tuple, set)):
        raise ValueError('Expected type in {list, tuple, set, str} but got ' + str(type(paths)))
    image_names = list()
    for path in paths:
        image_names += list(map(lambda x: x.split('.', 1)[0], os.listdir(path)))
    return df_class[df_class.image_name.map(lambda x: x in image_names)].reset_index(drop=True)

In [15]:
df_train = load_split(train_paths, df_class)
df_val = load_split(val_paths, df_class)

In [16]:
transform_train, transform_val = get_transforms()
ds_train = AmazonDataset(df_train, transform_train, paths=train_paths)
ds_val = AmazonDataset(df_val, transform_val, paths=val_paths)

### Model definition

We download directly weights from the pretrained resnet18, and freeze all weights. We overwrite the last fully connected layer, by adding two dense layers followed by a sigmoid. This `fc` last part is the only layer to be trained.

In [17]:
def get_model(pretrained=False, dropout_rate=0.2):
    model = models.resnet18(pretrained=pretrained)
    for param in model.parameters():
        param.require_grad = False
    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
    model.fc = nn.Sequential(
      nn.Flatten(),
      nn.Linear(512, 128), # 512 for resnet18 or 2048 for resnet 50
      nn.ReLU(inplace=True),
      nn.Dropout(dropout_rate),
      ######### FILL IN HERE #########
      nn.Linear(128, 17),
      ######### END #########
      nn.Sigmoid()
    )

    return model.to(device)

## Training

In [18]:
def train_batch(X, Y, model, loss_fn, optimizer):
    model.train()
    ######### FILL IN HERE #########

    Y_hat = model(X)

    # Step 2: Calculate loss
    batch_loss = loss_fn(Y_hat, Y)

    # Step 3: Backpropagation and parameter update
    optimizer.zero_grad()
    batch_loss.backward()
    optimizer.step()

    ######### END #########
    Y_hat = Y_hat.detach().float().cpu().numpy()  # detach predictions from gradients for logging

    return Y_hat, batch_loss.item()


@torch.no_grad()
def compute_val_loss(X, Y, model, loss_fn):
    model.eval()
    ######### FILL IN HERE #########

    Y_hat = model(X)

    # Step 2: Calculate loss
    batch_loss = loss_fn(Y_hat, Y)

    ######### END #########
    Y_hat = Y_hat.float().cpu().numpy()

    return Y_hat, batch_loss.item()

We choose to train our model for N epochs, while decreasing our learning rate by 10x every 7 batches. We monitor the validation loss are our key metrics. The validation score is just useful as side indication, because we choose the classification threshold quite randomly (0.2). <br>
Later on, we will find the most adapted threshold for each target.

In [19]:
def train_model(model, optimizer, loss_fn, dl_train, dl_val, epochs, filename):
    lr_scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

    loss_train, loss_val = [], []
    score_train, score_val = [], []

    Y_hat_val = None
    best_loss_val = np.inf

    for idx in range(epochs):
        loss_train_epoch, loss_val_epoch = [], []
        Y_hat_train_epoch, Y_hat_val_epoch = [], []
        Y_train_epoch, Y_val_epoch = [], []

        for X, Y in tqdm(dl_train, leave=False):
            Y_hat, batch_loss = train_batch(X, Y, model, loss_fn, optimizer)
            loss_train_epoch.append(batch_loss)
            Y_hat_train_epoch.extend(Y_hat)
            Y_train_epoch.extend(Y.detach().float().cpu().numpy())

        for X, Y in tqdm(dl_val, leave=False):
            Y_hat, batch_loss = compute_val_loss(X, Y, model, loss_fn)
            loss_val_epoch.append(batch_loss)
            Y_hat_val_epoch.extend(Y_hat)
            Y_val_epoch.extend(Y.detach().float().cpu().numpy())

        avg_loss_train = np.mean(loss_train_epoch)
        avg_loss_val = np.mean(loss_val_epoch)

        Y_hat_train_epoch = np.array(Y_hat_train_epoch)
        Y_hat_val_epoch = np.array(Y_hat_val_epoch)
        Y_thresh_train_epoch = (Y_hat_train_epoch > .2).astype(float)
        Y_thresh_val_epoch = (Y_hat_val_epoch > .2).astype(float)
        Y_train_epoch = np.array(Y_train_epoch)
        Y_val_epoch = np.array(Y_val_epoch)

        score_train_epoch = fbeta_score(Y_train_epoch, Y_thresh_train_epoch, beta=2, average="samples")
        score_val_epoch = fbeta_score(Y_val_epoch, Y_thresh_val_epoch, beta=2, average="samples")

        # saving values for debugging
        if avg_loss_val < best_loss_val:
            best_loss_val = avg_loss_val
            Y_hat_val = Y_hat_val_epoch
            Y_thresh_val = Y_thresh_val_epoch
            Y_val = Y_val_epoch

        loss_train.append(avg_loss_train)
        loss_val.append(avg_loss_val)
        score_train.append(score_train_epoch)
        score_val.append(score_val_epoch)

        print(
            f"epoch: {idx}/{epochs} -- train loss: {avg_loss_train}, " \
            f"val loss: {avg_loss_val}" \
            f" -- train fbeta_score: {score_train_epoch}, " \
            f"val fbeta_score: {score_val_epoch}"
        )

        lr_scheduler.step()

    train_results = {
        "loss_train": loss_train,
        "loss_val": loss_val,
        "score_train": score_train,
        "score_val": score_val,
        "Y_hat_val": Y_hat_val,
        "Y_thresh_val": Y_thresh_val,
        "Y_val": Y_val,
    }

    torch.save(model, filename + '.pt')
    pickle.dump(train_results, open(filename+'.results', "wb"))
    print(f'Saved model to {filename}.pt')
    print(f'Saved results to {filename}.results')

There is a risk that we have created an imbalance within our rare classes, by splitting randomly our dataset between train and validation. Let's ensure that rare classes representation are similar between both datasets.

In [20]:
RARE_CLASSES = [
    "bare_ground", "selective_logging", "artisinal_mine", "blooming", "slash_burn", "blow_down", "conventional_mine"
]

In [21]:
for rare_class in RARE_CLASSES:
    total_train = df_train.loc[df_train.tags.str.contains(rare_class)].shape[0]
    total_val = df_val.loc[df_val.tags.str.contains(rare_class)].shape[0]
    print(f"train {rare_class}: {100 * total_train / df_train.shape[0]:.4f}% ({total_train})")
    print(f"val {rare_class}: {100 * total_val / df_val.shape[0]:.4f}% ({total_val})\n")

train bare_ground: 2.1533% (229)
val bare_ground: 2.1410% (174)

train selective_logging: 0.7616% (81)
val selective_logging: 0.8244% (67)

train artisinal_mine: 0.7710% (82)
val artisinal_mine: 0.9228% (75)

train blooming: 0.7052% (75)
val blooming: 0.8121% (66)

train slash_burn: 0.5078% (54)
val slash_burn: 0.5414% (44)

train blow_down: 0.2445% (26)
val blow_down: 0.2584% (21)

train conventional_mine: 0.2633% (28)
val conventional_mine: 0.1600% (13)



In [22]:
# Set hyeprparameters for batch size, epochs, learning rate
BATCH_SIZE = 64
LR = 1e-4
EPOCHS = 10

# Load dataset in DataLoader
dl_train = DataLoader(
    ds_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=ds_train.collate_fn
)
dl_val = DataLoader(
    ds_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=ds_val.collate_fn
)

# Initialize model
######### FILL IN HERE #########
# What needs to change to use the pretrained model?
model = get_model(pretrained=True)
######### END #########
optimizer = Adam(model.parameters(), lr=LR)
loss_fn = nn.BCELoss()

# Checkpoint file to save models + results
filename = f'/content/drive/MyDrive/pretrained_resnet18__batch_size={BATCH_SIZE}__lr={LR}__epochs={EPOCHS}'
train_model(model, optimizer, loss_fn, dl_train, dl_val, EPOCHS, filename)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 238MB/s]


  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

epoch: 0/10 -- train loss: 0.22232262589438948, val loss: 0.11847140505088596 -- train fbeta_score: 0.790340051501803, val fbeta_score: 0.8984177564156477


  0%|          | 0/167 [00:00<?, ?it/s]

...

  0%|          | 0/127 [00:00<?, ?it/s]

epoch: 8/10 -- train loss: 0.018429446653156222, val loss: 0.12044614265988192 -- train fbeta_score: 0.9887465686119933, val fbeta_score: 0.9069808303153208


  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/127 [00:00<?, ?it/s]

epoch: 9/10 -- train loss: 0.016992865252994493, val loss: 0.12106753498550475 -- train fbeta_score: 0.9904754479889184, val fbeta_score: 0.906750478773711
Saved model to /content/drive/MyDrive/pretrained_resnet18__batch_size=64__lr=0.0001__epochs=10.pt
Saved results to /content/drive/MyDrive/pretrained_resnet18__batch_size=64__lr=0.0001__epochs=10.results


In [23]:
model = torch.load(filename + '.pt')  # load best model
train_results = pickle.load(open(filename + '.results', "rb"))  # load best model results

In [24]:
# Track loss and performance over time (for train and val)
loss_train = train_results["loss_train"]
loss_val = train_results["loss_val"]
score_train = train_results["score_train"]
score_val = train_results["score_val"]

fig = make_subplots(rows=1, cols=2, subplot_titles=("Loss", "Fbeta scores"))
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_train))),
        y=loss_train,
        name="loss_train",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(loss_val))),
        y=loss_val,
        name="loss_val",
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_train))),
        y=score_train,
        name="score_train",
    ),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(
        x=list(range(len(score_val))),
        y=score_val,
        name="score_val",
    ),
    row=1, col=2
)
fig.show()

# Understanding results

### Marginal proba

Let display the average of our Y_hat prediction proba, when the truth is Y = 1 or Y = 0.

In [25]:
Y_hat_val = np.array(train_results["Y_hat_val"])
Y_val = np.array(train_results["Y_val"])

pos_probas, neg_probas = [], []
for class_, idx in LABEL_IDXS.items():
    pos_probas.append(Y_hat_val[np.where(Y_val[:, idx] != 0), idx].mean())
    neg_probas.append(Y_hat_val[np.where(Y_val[:, idx] == 0), idx].mean())
go.Figure([
    go.Bar(x=list(LABEL_IDXS), y=pos_probas, name="Y_hat proba | Y = 1"),
    go.Bar(x=list(LABEL_IDXS), y=neg_probas, name="Y_hat proba | Y = 0")
]).show()

Nice. We see that default thresholding at .2 might not always work. So we need to define the best threshold for each individual classes.

In [26]:
def find_best_thresholds(Y_hat, Y):
    """
    This is just a simple solution, but there are more rigorous ways to choose this threshold.
    """
    N_tags = Y.shape[1]
    best_threshs = [0.2] * N_tags
    resolution = 100
    for jdx in tqdm(range(N_tags)):
        best_score = 0
        #threshs = np.zeros_like(best_threshs)
        threshs = best_threshs.copy()
        for kdx in range(resolution):
            kdx /= resolution
            threshs[jdx] = kdx
            Y_hat_thresh = (Y_hat > threshs).astype(float)
            score = fbeta_score(Y, Y_hat_thresh, beta=2, average="samples")
            if score > best_score:
                best_score = score
                best_threshs[jdx] = kdx

    global_best_score = fbeta_score(Y, (Y_hat > best_threshs).astype(float), beta=2, average="samples")
    print(f"threshs: {best_threshs} -- best score: {global_best_score}")

    return best_threshs

In [ ]:
threshs = find_best_thresholds(Y_hat_val, Y_val)

  0%|          | 0/17 [00:00<?, ?it/s]

We improvemed our validaiton score slightly. Now let's observe the score by class.

In [ ]:
class_scores = {}
classes = list(LABEL_IDXS.keys())
for jdx in range(Y_val.shape[1]):
    y_val = Y_val[:, jdx].ravel()
    y_hat_val = (Y_hat_val[:, jdx].ravel() > threshs[jdx]).astype(float)
    score = fbeta_score(y_val, y_hat_val, beta=2)
    class_scores[classes[jdx]] = round(score, 4)

df_score = pd.DataFrame(dict(
    label=list(class_scores.keys()), score=list(class_scores.values()),
)).sort_values("score", ascending=False)
fig = px.bar(df_score, x="label", y="score", color="score")
fig.show()

Half of our rare classes are below the 50% score. There is room for improvement here. Let's observe all confusion matrices for each classes.

### Confusion matrixes

In [ ]:
fig = make_subplots(cols=5, rows=4)
for jdx in range(Y_val.shape[1]):
    y_val = Y_val[:, jdx].ravel()
    y_hat_val = (Y_hat_val[:, jdx].ravel() > threshs[jdx]).astype(float)
    tn, fp, fn, tp = confusion_matrix(y_val, y_hat_val).ravel()
    mat = np.array([[fn, tn], [tp, fp]])
    col = jdx // 4+1
    row = jdx % 4+1
    fig.add_trace(
        go.Heatmap(
            z=mat, text=[[f"fn: {fn}", f"tn: {tn}"], [f"tp: {tp}", f"fp: {fp}"]],
            texttemplate="%{text}", colorscale='Viridis', name=list(LABEL_IDXS.keys())[jdx],
            showscale=False
        ),
        col=col, row=row,
    )
    fig.update_xaxes(title=list(LABEL_IDXS.keys())[jdx], showticklabels=False, row=row, col=col)
    fig.update_yaxes(showticklabels=False, row=row, col=col)

fig.update_layout(
    width=1200, height=800, title="Confusion matrices",
)
fig.show()

- `slash_burn` and `blowdown` scores contains more false negative (FN) than false positive (FP)
- However, fbeta score is more sensitive to FP than FN by design.
- Because we choose to shuffle our dataset in our dataloader, we can't display the original image of these fp and fn to visually understand our model errors. That's an issue of our implementation. So to furthering analysis, we might return filenames in the `__getitem__` and `collate_fn` methods of our custom Dataset class.